# Projet maison n° 4

In [1]:
# imports
import requests
import re
import urllib.request, json 
from bs4 import BeautifulSoup

**Partie A**

Ecrire une fonction `get_prices_from_url()` qui extrait des informations à partir des 2 pages ci-dessous.

```python
URL_PAGE2 = "https://kim.fspot.org/cours/page2.html"
URL_PAGE3 = "https://kim.fspot.org/cours/page3.html"
```

Avec `URL_PAGE2`, la fonction doit retourner :

```json
{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
 'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
 'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}}
```

In [2]:
# partie A
URL_PAGE2 = "https://kim.fspot.org/cours/page2.html"
URL_PAGE3 = "https://kim.fspot.org/cours/page3.html"

def get_prices_from_url(url):
    prices = {}
    html_content = requests.get(url).text
    soup = BeautifulSoup(html_content, "lxml")
    
    for tag in soup.findAll('div', attrs={'class': 'pricing-tables pure-g'}):
        t_headers = []
        for plan in tag.findAll('h2'):
            t_headers.append(plan.text)
    
        price_data = []
        for price in tag.findAll('span', attrs={'class' : 'pricing-table-price'}):
            price_data.append(str(price.text).strip()[:-10])
    
        storage_data = []
        database_data = []
        for li in tag.findAll('li'):
            storage = re.search(r'([0-9]+[GBT]*) file storage', li.text.strip())
            database = re.search(r'([0-9]+) database', li.text.strip())
            if storage:
                storage_data.append(storage.group(1))
            if database:
                database_data.append(database.group(1))
    
    for i, header in enumerate(t_headers):
        prices[header] = {'price' : price_data[i],
                          'storage' : storage_data[i],
                          'databases' : int(database_data[i])}
    
    return prices

**Partie B**

*L'abus d'alcool est dangereux pour la santé, à consommer avec modération.*

1) Ecrire une fonction `extract_beer_infos()` qui extrait des informations sur une bière du site de bières *beowulf*.

Exemple d'URL: https://www.beerwulf.com/fr-fr/p/bieres/cuvee-des-trolls.33 

La fonction doit retourner :
```json
{'Nom': 'Cuvée des Trolls',
 'Style': 'Bière Blonde',
 'Contenu': 25,
 'Degré d’alcool': 7.0,
 'Origine': 'Belgique',
 'Brasseur': 'Brasserie Dubuisson Freres'}
```

2) L'URL ci-après retourne un JSON avec une liste de bières :

```python
URL_BEERLIST_FRANCE = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France&container=Bouteille"
```

Ecrire une fonction `extract_beer_list_infos(url)` qui prend en argument cet URL et retourne les informations sur une liste de bières du site *beowulf*.

Cette fonction doit retourner la liste des informations obtenues par la fonction précédemment définie `extract_beer_infos()`.

Exemple de retour :

```json
[{'Nom':'Desperados','Style':'Lager','Contenu':33,'Degré d’alcool':5.9,'Origine':'France','Brasseur':'Desperados'},
{'Nom':'La Lager Sans Gluten de Vézelay','Style':'Lager','Contenu':25,'Degré d’alcool':4.0,'Origine':'France','Brasseur':'Brasserie de Vézelay'},
{'Nom':'Mélusine Bio','Style':'Pale Ale','Contenu': 33,'Degré d’alcool': 5.0,'Origine':'France','Brasseur':'Mélusine'},
{'Nom':'La Parisienne Le Titi Parisien','Style':'IPA','Contenu':33,'Degré d’alcool':5.5,'Origine':'France','Brasseur': 'Brasserie la Parisienne'},
{'Nom':'Brasserie De Sutter Brin de Folie','Style':'Bière Blonde','Contenu': 33,'Degré d’alcool':6.5,'Origine':'France','Brasseur':'Brasserie de Sutter'}]
```

**Facultatif**

Chercher comment optimiser cette fonction en utilisant `multiprocessing.Pool()` pour paralléliser les accès web.

In [3]:
# partie B-1
def extract_beer_infos(url):
    html_content = requests.get(url).text
    soup = BeautifulSoup(html_content, "lxml")
    
    for tag in soup.findAll('div', attrs={'class': 'small-12 content-column'}):
        name = ""
        for name in tag.findAll('div', attrs={'class': 'product-detail-info-title'}):
            name = name.text.strip()
    
        for product_info in tag.findAll('div', attrs={'class': 'product-info beer-product-info'}):
            style = ""
            for style in product_info.findAll('a'):
                style = style.text.strip()
        
            contenu = 0
            for contenu in product_info.findAll('dd', attrs={'class': 'small-6 medium-9 columns js-beer-volume'}):
                contenu = int(contenu.text.strip()[0:2])
            
            alcool = 0.0
            for degree_alcool in product_info.findAll('dd'):
                degree = re.search(r'([0-9]+,[0-9]+)%', degree_alcool.text.strip())
                if degree:
                    alcool = float(degree.group(1).replace(',', '.'))
                
            origine = ""
            for pays in product_info.findAll('dd', attrs={'class': 'small-6 medium-9 columns js-beer-country'}):
                origine = pays.text.strip()
            
            brasseur = ""
            brasseur = product_info.findAll('dd')[4].text.strip()
    
    beer_infos = {
        'Nom': name,
        'Style': style,
        'Contenu': contenu,
        'Degré d’alcool': alcool,
        'Origine': origine,
        'Brasseur': brasseur
    }
    
    return beer_infos

In [4]:
# partie B-2
#URL_BEERLIST_FRANCE = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France&container=Bouteille"
URL_BEERLIST_FRANCE = 'https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France'
def extract_beer_list_infos(url):
    # Collecter les pages de bières à partir du JSON
    beer_pages = []
    
    r = requests.get(URL_BEERLIST_FRANCE)
    json = r.json()
    
    for item in json['items']:
        beer_pages.append(item['contentReference'])
    
    # Sequential version (slow):
    beers = []
    
    for page in beer_pages:
        beers.append(extract_beer_infos('https://www.beerwulf.com/' + page))

    # Facultatif
    # Parallel version (faster):
    # beers = []
    
    return beers

In [5]:
import unittest

class Session4Tests(unittest.TestCase):
    def test_01_get_prices_from_url_page2(self):
        prices = get_prices_from_url(URL_PAGE2)
        # We should have found 3 products:
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 3)
        self.assertIn('Personal', prices)
        self.assertIn('Small Business', prices)
        self.assertIn('Enterprise', prices)
        
        personal = prices['Personal']
        self.assertIn('price', personal)
        self.assertIn('storage', personal)
        self.assertIn('databases', personal)
        self.assertEqual(personal['price'], '$5')
        self.assertEqual(personal['storage'], '1GB')
        self.assertEqual(personal['databases'], 1)
        
    def test_02_get_prices_from_url_page3(self):
        prices = get_prices_from_url(URL_PAGE3)
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 4)
        self.assertEqual(
            prices['Privilege'],
            {'databases': 100, 'price': '$99', 'storage': '1TB'}
        )
    
    def test_03_extract_beer_list_infos(self):
        infos = extract_beer_list_infos(URL_BEERLIST_FRANCE)
        self.assertIsInstance(infos, list)
        self.assertGreater(len(infos), 1)
        # Contenu = int
        # Degré d’alcool = float
        for beer in infos:
            self.assertIsInstance(beer['Nom'], str)
            self.assertIsInstance(beer['Style'], str)
            self.assertIsInstance(beer['Contenu'], int)
            self.assertIsInstance(beer['Degré d’alcool'], float)
            self.assertEqual(beer['Origine'], "France")
            self.assertIsInstance(beer['Brasseur'], str)

            
def run_tests():
    test_suite = unittest.makeSuite(Session4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [6]:
if __name__ == '__main__':
    run_tests()

test_01_get_prices_from_url_page2 (__main__.Session4Tests) ... ok
test_02_get_prices_from_url_page3 (__main__.Session4Tests) ... ok
test_03_extract_beer_list_infos (__main__.Session4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 2.688s

OK
